# Idiot
https://habrahabr.ru/post/342738/

In [26]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf
from dataset import Dataset

In [27]:
dataset = Dataset()
#dataset.load('data/dostoevsky-all.txt')
dataset.load('data/anna.txt')
model_path = 'models/002/model.ckpt'

In [28]:
def build_inputs():
    inputs  = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

def build_lstm(lstm_size, num_layers, keep_prob):
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    return cell

def build_output(lstm_output, lstm_size, out_size):
    logits = tf.layers.dense(lstm_output, out_size)
    out = tf.nn.softmax(logits, name='predictions')
    return out, logits

def build_loss(logits, targets, lstm_size, num_classes):
    loss = tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, 
        labels=tf.one_hot(targets, num_classes))
    loss = tf.reduce_mean(loss)
    return loss

def build_optimizer(loss, learning_rate, grad_clip):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [29]:
class CharRNN:
    
    def __init__(self, num_classes, lstm_size=128, num_layers=2, learning_rate=0.001, grad_clip=5):

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs()

        # Строим LSTM ячейку
        cell = build_lstm(lstm_size, num_layers, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        embed = tf.one_hot(self.inputs, num_classes)
        
        
        # Прогоняем данные через RNN и собираем результаты
        self.seq_length_pl = tf.placeholder(tf.int32, [None], 'seq_length')
        self.batch_size_pl = tf.placeholder(tf.int32, shape=[], name='batch_size')
        self.initial_state = cell.zero_state(self.batch_size_pl, tf.float32)
        outputs, state = tf.nn.dynamic_rnn(
            cell, 
            embed,
            sequence_length=self.seq_length_pl, 
            initial_state = self.initial_state,
            dtype=tf.float32)
        self.final_state = state
        self.cell = cell

        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [30]:
batch_size    = 100        # Размер пакета
num_steps     = 100         # Шагов в пакете
lstm_size     = 512         # Количество LSTM юнитов в скрытом слое
num_layers    = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob     = 0.5         # Dropout keep probability

vocab = dataset.vocab

In [31]:
model = CharRNN(
    len(vocab), 
    lstm_size=lstm_size, num_layers=num_layers, 
    learning_rate=learning_rate
)

In [ ]:
%%time
epochs = 50
# Сохраняться каждый N итераций
save_every_n = 100

saver = tf.train.Saver()
sess  = tf.Session()
try:
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, model_path+"xxx")
    except:
        print("no data at", model_path)
    counter = 0
    for e in range(epochs):
        loss = 0
        new_state = sess.run(model.initial_state, feed_dict={model.batch_size_pl: batch_size})

        for x, y in dataset.get_batches(batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.seq_length_pl: [num_steps, ]*batch_size,
                    model.initial_state: new_state,
                    model.keep_prob: keep_prob}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)

            end = time.time()

            if (counter % save_every_n == 0):
                saver.save(sess, model_path)
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
except KeyboardInterrupt:
    pass

INFO:tensorflow:Restoring parameters from models/002/model.ckptxxx


INFO:tensorflow:Restoring parameters from models/002/model.ckptxxx


no data at models/002/model.ckpt
Epoch: 1/50...  Training Step: 100...  Training loss: 3.2487...  0.3986 sec/batch


In [23]:
def pick_top_n(preds, vocab_size, top_n):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [24]:
def sample(model, n_samples, lstm_size, vocab_size, top_n, prime="Глаза его были большие"):
    samples = [c for c in prime]
    new_state = sess.run(model.initial_state, feed_dict={model.batch_size_pl: 1})
    for c in prime:
        x = np.zeros((1, 1))
        x[0,0] = dataset.vocab_to_int[c]
        feed = {model.inputs: x,
                model.seq_length_pl: [1],
                model.initial_state: new_state,
                model.keep_prob: 1.}
        preds, new_state = sess.run([model.prediction, model.final_state], feed_dict=feed)

    c = pick_top_n(preds, len(dataset.vocab), top_n)
    samples.append(dataset.int_to_vocab[c])

    for i in range(n_samples):
        x[0,0] = c
        feed = {model.inputs: x,
                model.seq_length_pl: [1],
                model.initial_state: new_state,
                model.keep_prob: 1.}
        preds, new_state = sess.run([model.prediction, model.final_state], feed_dict=feed)

        c = pick_top_n(preds, len(dataset.vocab), top_n)
        samples.append(dataset.int_to_vocab[c])
        
    return ''.join(samples)

In [25]:
# samp = sample(1000, lstm_size, len(vocab), top_n=5, prime=)
primes = ['Быть или не быть?']
for prime in primes:
    print('-'*50)
    samp = sample(model, 500, lstm_size, len(dataset.vocab), top_n=5, prime=prime)
    samp = samp.replace('\n', '\n\n')
    print(samp)

--------------------------------------------------
Доколе математики будут уверены в непогрешимости своего мозга, и доказательств, оттуда выходящих? Как было не большо на приета. Он не приеда и ничего носчаство плитать в свойй. Но он, он всегда. Как на стольным сторо не все дажи положить его, что он с пруговорили, они был начал свижениеми своего не отвечали, и начего на нее, он, остановила, которого был брать на ним, крого воспотия, на востовое соберенностими, и она всказал на слуша. И с кругим и ниходо составленное в светем, напрясиния на нее. И потому что ты споловали в этом подержавшие просто с них не скость.



Степан Ииа, Александрович по
